# Day 7 - Lab 2: Building Agent Frontends with A2A (Solution)

**Objective:** Use the A2A (Ask-Advise-Act) Framework to build a guided, conversational user interface for an AI agent.

**Introduction:**
This solution notebook provides the complete, runnable Python script for the Streamlit application. It includes the full prompts for each phase of the A2A workflow and explains how they work together to create a guided experience for the user.

## Step 1: Setup

**Explanation:**
This lab is unique in that it's a standalone Python script, not a Jupyter notebook. The code below is the complete content for the `sql_query_generator.py` file. We use the `save_artifact` helper to write this file into the correct lab directory.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from utils import save_artifact

sql_app_code = """ 
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize the LLM client
try:
    llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))
except Exception as e:
    st.error(f"Failed to initialize LLM. Check your API key. Error: {e}")
    st.stop()

st.title("SQL Query Generator with A2A")

# --- UI Setup ---
st.header("1. Provide Context")
schema = st.text_area("Database Schema", height=150, placeholder="CREATE TABLE customers (id INT, name VARCHAR(255), state VARCHAR(2));")
goal = st.text_area("What is your goal?", placeholder="Show me all customers in California ordered by last name.")

# --- Session State Initialization ---
if 'step' not in st.session_state:
    st.session_state.step = "START"
if 'questions' not in st.session_state:
    st.session_state.questions = []
if 'answers' not in st.session_state:
    st.session_state.answers = []
if 'plan' not in st.session_state:
    st.session_state.plan = ""
if 'query' not in st.session_state:
    st.session_state.query = ""

# --- A2A Workflow --- 

if st.button("Begin Query Generation", key="start_button"):
    if schema and goal:
        st.session_state.step = "ASK"
        # --- ASK PHASE ---
        with st.spinner("Asking clarifying questions..."):
            ask_prompt = f'''You are an AI assistant helping a user write a SQL query. Your first step is to ask clarifying questions to better understand the user's needs.\n\n**Database Schema:**\n{schema}\n\n**User's Goal:**\n{goal}\n\nBased on the schema and goal, generate 1-2 brief clarifying questions that would help you write a more accurate query. For example, ask about ordering or filtering. Output ONLY the questions, separated by a question mark.'''
            response = llm.invoke(ask_prompt).content
            st.session_state.questions = [q.strip() for q in response.split('?') if q.strip()]
            st.session_state.answers = ["" for _ in st.session_state.questions]
            st.session_state.plan = ""
            st.session_state.query = ""
    else:
        st.error("Please provide both a schema and a goal.")

if st.session_state.step == "ASK" and st.session_state.questions:
    st.header("2. Ask: Clarifying Questions")
    for i, q in enumerate(st.session_state.questions):
        st.session_state.answers[i] = st.text_input(f"{q}?", key=f"ans{i}")
    
    if st.button("Submit Answers"):
        st.session_state.step = "ADVISE"
        # --- ADVISE PHASE ---
        with st.spinner("Creating a query plan..."):
            clarifications = "\n".join([f"- Q: {q}? A: {a}" for q, a in zip(st.session_state.questions, st.session_state.answers) if a])
            advise_prompt = f'''You are an AI assistant. Your goal is to create a step-by-step plan to write a SQL query.\n\n**Database Schema:**\n{schema}\n\n**User's Goal:**\n{goal}\n\n**User's Answers to Clarifying Questions:**\n{clarifications}\n\nBased on all this information, create a numbered, step-by-step plan for constructing the SQL query. For example: 1. SELECT the required columns. 2. FROM the correct table. 3. WHERE the state is 'California'. 4. ORDER BY the last name.'''
            st.session_state.plan = llm.invoke(advise_prompt).content
        st.experimental_rerun()

if st.session_state.step == "ADVISE" and st.session_state.plan:
    st.header("3. Advise: Query Plan")
    st.markdown(st.session_state.plan)
    if st.button("Generate SQL from Plan"):
        st.session_state.step = "ACT"
        # --- ACT PHASE ---
        with st.spinner("Generating final SQL query..."):
            clarifications = "\n".join([f"- Q: {q}? A: {a}" for q, a in zip(st.session_state.questions, st.session_state.answers) if a])
            act_prompt = f'''You are a SQL expert. Your task is to write a single, valid SQL query based on the provided plan and context. Output ONLY the raw SQL query inside a markdown code block.\n\n**Database Schema:**\n{schema}\n\n**User's Goal:**\n{goal}\n\n**Clarifications:**\n{clarifications}\n\n**Execution Plan:**\n{st.session_state.plan}'''
            query = llm.invoke(act_prompt).content
            if '```' in query:
                query = query.split('```')[1].lstrip('sql').strip()
            st.session_state.query = query
        st.experimental_rerun()

if st.session_state.step == "ACT" and st.session_state.query:
    st.header("4. Act: Final SQL Query")
    st.code(st.session_state.query, language='sql')
"""

save_artifact(sql_app_code, "labs/Day_07_Advanced_Agent_Workflows/sql_query_generator.py")
print("Saved 'sql_query_generator.py'. To run it, open your terminal and execute: streamlit run labs/Day_07_Advanced_Agent_Workflows/sql_query_generator.py")

## Lab Conclusion

Congratulations! You have built a practical application using the A2A framework to create a guided, conversational experience. This pattern is incredibly powerful for turning complex tasks (like writing SQL) into a simple, step-by-step process for users. You've learned how to break down a problem into the Ask, Advise, and Act phases to create more user-friendly and effective AI tools.